In [1]:
# https://towardsdatascience.com/pandas-groupby-a-simple-but-detailed-tutorial-314b8f37005d
# https://towardsdatascience.com/accessing-data-in-a-multiindex-dataframe-in-pandas-569e8767201d
# https://towardsdatascience.com/summarizing-data-with-pandas-crosstab-efc8b9abecf
# https://towardsdatascience.com/how-to-flatten-multiindex-columns-and-rows-in-pandas-f5406c50e569
# https://datascientyst.com/list-aggregation-functions-aggfunc-groupby-pandas/
# https://stackoverflow.com/questions/25929319/how-to-iterate-over-pandas-multiindex-dataframe-using-index
# https://stackoverflow.com/questions/24495695/pandas-get-unique-multiindex-level-values-by-label
# https://stackoverflow.com/questions/55706391/pandas-crosstab-on-multiple-columns-then-groupby

# https://matplotlib.org/stable/gallery/pyplots/pyplot_text.html#sphx-glr-gallery-pyplots-pyplot-text-py

In [2]:
import pandas as pd
import numpy as np
from myUtils import pickle_load, pickle_dump

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

# # filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
# filename_symbols = path_data_dump + 'my_symbols.csv'  # symbols text file

# _filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV
filename_pickled_perf_rank_dict =  'perf_rank_dict'  # store symbols from performance rank results
filename_pickled_r_all_ranks =  'r_all_ranks'  # list of top 100 most common symbols from performance rank results
filename_pickled_df_a = 'df_OHLCV_clean'  # df adjusted OHLCV, dropped symbols with no vol and close
filename_pickled_df_c = 'df_close_clean'  # df close, dropped symbols with no vol and close

verbose = False  # True prints more output

#################
# look_back_days = -250 * 60  # subset df iloc days
look_back_days = -250 * 6  # subset df iloc days, 6 years of data
#################

In [3]:
print(f"Full path to pickled df_symbols_close:  {path_data_dump}{filename_pickled_df_symbols_close}")
df_close = pickle_load(path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)
print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
df_adjOHLCV = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)

Full path to pickled df_symbols_close:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_symbols_close
Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV


In [4]:
# https://stackoverflow.com/questions/63826291/pandas-series-find-column-by-value
df = df_adjOHLCV[look_back_days::]
df_v = df.xs('Volume', level=1, axis=1)  # select only Volume columns
rows, cols = np.where(df_v == 0)  # row index, column index where trading volumes are zero
idx_no_volume = list(set(cols))
idx_no_volume.sort()
symbols_no_volume = df_v.columns[idx_no_volume]
print(f'symbols with no volume:\n{symbols_no_volume}')

symbols with no volume:
Index(['AMCR', 'AVT', 'BIIB', 'BKR', 'CCI', 'CHD', 'CNC', 'CNXC', 'DISH',
       'ESAB', 'FCFS', 'GEN', 'IBKR', 'ICPT', 'JJC', 'PEAK', 'PFG', 'SBNY',
       'SGEN', 'SRPT', 'SSB', 'VC', 'VMW', 'VNT', 'WH', 'XEL'],
      dtype='object')


In [5]:
df_dif = df_v - df_v.shift(periods=1)
rows, cols = np.where(df_dif == 0)
idx_same_volume = list(set(cols))
idx_same_volume.sort()
idx_same_volume
symbols_same_volume = df_v.columns[idx_same_volume]
print(f'symbols with same volume:\n{symbols_same_volume}')

symbols with same volume:
Index(['ABM', 'ACM', 'ACN', 'ADP', 'AGCO', 'ALRM', 'AMCR', 'AMED', 'AMG',
       'AMGN',
       ...
       'VRE', 'VRSK', 'VSAT', 'WERN', 'WEX', 'WSFS', 'WTS', 'WTW', 'XEL',
       'XENE'],
      dtype='object', length=250)


In [6]:
df_c = df.xs('Close', level=1, axis=1)  # select only Close columns
df_c = df_c.fillna(0).copy()  # convert NaNs to zeros
rows, cols = np.where(df_c == 0)  # row index, column index where trading volumes are zero
idx_no_close = list(set(cols))
idx_no_close.sort()
symbols_no_close = df_c.columns[idx_no_close]
print(f'symbols with NaN close:\n{symbols_no_close}')

symbols with NaN close:
Index(['ABNB', 'AIRC', 'ALGM', 'BJ', 'BRBR', 'CARR', 'CDAY', 'CEG', 'CHK',
       'CHX', 'CNXC', 'CR', 'CRWD', 'CTVA', 'DDOG', 'DISH', 'DOW', 'DT', 'DTM',
       'ESAB', 'ETRN', 'FOX', 'FOXA', 'FYBR', 'GEHC', 'GFS', 'GO', 'GXO',
       'HEP', 'ICPT', 'JJC', 'KD', 'LCID', 'MP', 'MRNA', 'NARI', 'NVST', 'NVT',
       'OGN', 'OTIS', 'PACW', 'PDD', 'PGNY', 'RIVN', 'SGEN', 'SGOV', 'SHC',
       'SITM', 'SWAV', 'VMW', 'VNT', 'VSCO', 'WH', 'YETI', 'ZM', 'ZS'],
      dtype='object')


In [7]:
symbols_bad_data = list(symbols_no_close) + list(symbols_no_volume) + list(symbols_same_volume) # combine symbols with no volume and no close
unique_symbols_bad_data = sorted(list(set(symbols_bad_data)))  # unique symbols
print(f'unique symbols with bad data, e.g. no volume, same volume and $0 close, includes duplicate symbols: {len(unique_symbols_bad_data)}')

unique symbols with bad data, e.g. no volume, same volume and $0 close, includes duplicate symbols: 307


In [8]:
# get symbols of past model picks
df_picks = pickle_load(path_data_dump, 'df_picks', verbose=verbose)
df_picks.head(1)

,date_end_df_train,max_days_lookbacks,days_lookbacks,sym_freq_15,sym_freq_14,sym_freq_13,sym_freq_12,sym_freq_11,sym_freq_10,sym_freq_9,sym_freq_8,sym_freq_7,sym_freq_6,sym_freq_5,sym_freq_4,sym_freq_3,sym_freq_2
0,2024-01-08,120,"[30, 60, 120]",[],[],[],[],[],[],['CYTK'],"['FTSM', 'SHV']",[],"['GBTC', 'GPS', 'HA']","['MBI', 'X']","['ANF', 'FATE', 'HY', 'MARA']","['BTC-USD', 'DXCM', 'ENV', 'FFWM', 'HOLI', 'HP...",[]


In [9]:
# keep only columns with symbols of past picks
df_picks.drop(columns=["date_end_df_train", "max_days_lookbacks", "days_lookbacks"], inplace=True)
df_picks.head(1)

,sym_freq_15,sym_freq_14,sym_freq_13,sym_freq_12,sym_freq_11,sym_freq_10,sym_freq_9,sym_freq_8,sym_freq_7,sym_freq_6,sym_freq_5,sym_freq_4,sym_freq_3,sym_freq_2
0,[],[],[],[],[],[],['CYTK'],"['FTSM', 'SHV']",[],"['GBTC', 'GPS', 'HA']","['MBI', 'X']","['ANF', 'FATE', 'HY', 'MARA']","['BTC-USD', 'DXCM', 'ENV', 'FFWM', 'HOLI', 'HP...",[]


In [10]:
df_picks.columns

Index(['sym_freq_15', 'sym_freq_14', 'sym_freq_13', 'sym_freq_12',
       'sym_freq_11', 'sym_freq_10', 'sym_freq_9', 'sym_freq_8', 'sym_freq_7',
       'sym_freq_6', 'sym_freq_5', 'sym_freq_4', 'sym_freq_3', 'sym_freq_2'],
      dtype='object')

In [11]:
import ast

In [12]:
list_of_lists = []

for col in df_picks.columns:
  # convert column values from string to list, e.g. '[]', '[]', '["A", "B", ..]' ... to [], [], ["A", "B", ..], ...
  l_series = df_picks[col].apply(ast.literal_eval)
  # list_of_lists = [l_item for l_item in l_series if l_item]  # this doesn't works  
  for l_item in l_series:
      if l_item:  # 
        list_of_lists.append(l_item)  

symbols_picks = [val for sublist in list_of_lists for val in sublist]
print(f'symbol count from model picks: {len(symbols_picks)}')
# list sorted unique symbols
unique_symbols_picks = sorted(list(set(symbols_picks)))
print(f'unique symbol count from model picks: {len(unique_symbols_picks)}')

symbol count from model picks: 55560
unique symbol count from model picks: 999


In [13]:
def find_common_items(list1, list2):
  """Finds the common items between two lists.

  Args:
    list1: The first list.
    list2: The second list.

  Returns:
    A list of the common items between the two lists.
  """

  common_items = []
  for item in list1:
    if item in list2:
      common_items.append(item)
  return common_items


# code in if block runs only from command line, code will NOT be executed if imported as a module
if __name__ == "__main__": 
  list1 = ["a", "b", "c", "d", "e"]
  list2 = ["b", "c", "d", "f", "g"]
  common_items = find_common_items(list1, list2)
  print(common_items)

['b', 'c', 'd']


In [14]:
# symbols are in past picks but also have bad data 
common_symbols = find_common_items(unique_symbols_picks, unique_symbols_bad_data)
print(common_symbols)

['AZPN', 'CHT', 'CMG', 'CVBF', 'CVLT', 'DISH', 'DVA', 'HEP', 'HOLI', 'HY', 'ICPT', 'MLKN', 'MLR', 'OSIS', 'PKX', 'PTMN', 'SFBS', 'SGEN', 'SRPT', 'TX', 'VNO', 'XENE']


In [15]:
def subtract_items(list1, list2):
  """Subtracts items in list2 from items in list1.

  Args:
    list1: The first list.
    list2: The second list.

  Returns:
    A list of the items in list1 that are not in list2.
  """

  subtracted_items = []
  for item in list1:
    if item not in list2:
      subtracted_items.append(item)
  return subtracted_items

In [16]:
symbols_drop = subtract_items(unique_symbols_bad_data, common_symbols)  # don't drop symbols in past picks
symbols_drop .sort()
print(f'len(unique_symbols_bad_data): {len(unique_symbols_bad_data)}')
print(f'len(common_symbols): {len(common_symbols)}')
print(f'len(symbols_drop): {len(symbols_drop)}')

len(unique_symbols_bad_data): 307
len(common_symbols): 22
len(symbols_drop): 285


In [17]:
# symbols_drop = list(symbols_no_close) + list(symbols_no_volume) + list(symbols_same_volume) # combine symbols with no volume and no close
# print(f'combined symbols with no volume, same volume and no close, inculdes duplicate symbols: {len(symbols_drop)}')
# symbols_drop = list(set(symbols_drop))  # drop duplicate symbols
# symbols_drop .sort()


df_a = df.drop(symbols_drop, axis=1, level=0)  # drop symbols from OHLCA df
df_c = df_close.iloc[look_back_days::]
df_c = df_c.drop(symbols_drop, axis=1)
print(f'unique symbols dropped from df_a (adjOLHLV) and df_c (Close): {len(symbols_drop)}')

unique symbols dropped from df_a (adjOLHLV) and df_c (Close): 285


In [18]:
print(f'symbols with no volume:   {len(symbols_no_volume):>5,}')
print(f'symbols with same volume: {len(symbols_same_volume):>5,}')
print(f'symbols with no close:    {len(symbols_no_close):>5,}\n')
print(f'symbols total before drop:                                        {len(df_close.columns):>5,}')
print(f'unique symbols dropped from df OHLCVA (df_a) and df Close (df_c): {len(symbols_drop):>5,}\n')
print('                                          symbols     rows')
print(f'df adjOHLCV (df_a) after dropped symbols:   {len(df_a.columns)/5:>5,.0f}    {len(df_a):>5,}')
print(f'df Close (df_c) after dropped symbols:      {len(df_c.columns):>5,}    {len(df_c):>5,}')


symbols with no volume:      26
symbols with same volume:   250
symbols with no close:       56

symbols total before drop:                                        1,552
unique symbols dropped from df OHLCVA (df_a) and df Close (df_c):   285

                                          symbols     rows
df adjOHLCV (df_a) after dropped symbols:   1,267    1,500
df Close (df_c) after dropped symbols:      1,267    1,500


In [19]:
pickle_dump(df_a, path_data_dump, filename_pickled_df_a)
print(f'pickled df adjOHLCV after dropping symbols with no volume, same volume, and no close:\n{path_data_dump}{filename_pickled_df_a}')
pickle_dump(df_c, path_data_dump, filename_pickled_df_c)
print(f'pickled df Close after dropping symbols with no volume, same volume, and no close:\n{path_data_dump}{filename_pickled_df_c}')

pickled df adjOHLCV after dropping symbols with no volume, same volume, and no close:
C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_OHLCV_clean
pickled df Close after dropping symbols with no volume, same volume, and no close:
C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_close_clean


In [20]:
from myUtils import list_dump

f_symbols_df_close_clean = 'symbols_df_close_clean.csv'  # symbols text file
symbols_df_c = list(df_c)  # column names in df_c
list_dump(symbols_df_c, path_data_dump, f_symbols_df_close_clean)# df_c.columns.to_csv(f_symbols_df_close_clean)

In [21]:
df_a.tail()

A                                                 \
                  Open        High         Low       Close     Volume   
Date                                                                    
2024-01-03  138.000000  138.000000  131.070007  131.160004  2074500.0   
2024-01-04  130.550003  131.500000  130.190002  131.000000  2446600.0   
2024-01-05  130.000000  131.960007  128.619995  130.559998  1394000.0   
2024-01-08  130.139999  133.570007  129.809998  133.380005  1311400.0   
2024-01-09  132.270004  135.645004  130.009995  130.679993  1409986.0   

                   AA                                              ...  \
                 Open       High        Low      Close     Volume  ...   
Date                                                               ...   
2024-01-03  32.119999  32.270000  31.045000  31.490000  8245700.0  ...   
2024-01-04  31.129999  31.490000  30.860001  31.129999  4543600.0  ...   
2024-01-05  30.540001  32.305000  30.090000  32.130001  7313000.0  ...   
2024-01-08  31.820000  32.090000  31.219999  31.730000  4992100.0  ...   
2024-01-09  31.820000  32.450001  31.549999  31.940001  5697097.0  ...   

                 ZUMZ                                                   ZWS  \
                 Open       High        Low      Close    Volume       Open   
Date                                                                          
2024-01-03  19.910000  20.170000  18.900000  18.930000  258400.0  28.950001   
2024-01-04  18.980000  18.980000  18.379999  18.600000  471800.0  28.190001   
2024-01-05  18.400000  18.850000  18.180000  18.290001  365700.0  28.100000   
2024-01-08  18.290001  18.780001  18.280001  18.770000  323200.0  27.910000   
2024-01-09  18.190001  18.500000  17.770000  18.209999  289934.0  27.709999   

                                                       
                 High        Low      Close    Volume  
Date                                                   
2024-01-03  29.070000  28.180000  28.180000  960100.0  
2024-01-04  28.660000  28.150000  28.320000  956400.0  
2024-01-05  28.459999  27.920000  27.930000  947500.0  
2024-01-08  28.059999  27.750000  28.049999  973900.0  
2024-01-09  28.069901  27.549999  28.040001  725649.0  

[5 rows x 6335 columns]

In [22]:
df_c.tail()

,A,AA,AAL,AAP,AAPL,AB,ABBV,ABR,ABT,ACGL,...,YY,ZBH,ZBRA,ZD,ZG,ZION,ZTO,ZTS,ZUMZ,ZWS
Date,,,,,,,,,,,,,,,,,,,,,
2024-01-03,131.160004,31.490000,12.95,61.000000,184.250000,30.370001,160.460007,14.69,109.519997,76.940002,...,32.970001,120.470001,252.520004,65.860001,53.980000,42.000000,21.059999,192.929993,18.930000,28.180000
2024-01-04,131.000000,31.129999,13.09,60.490002,181.910004,31.049999,161.460007,14.81,110.980003,77.470001,...,33.459999,120.199997,252.970001,65.570000,52.750000,42.639999,20.490000,194.039993,18.600000,28.320000
2024-01-05,130.559998,32.130001,13.60,61.520000,181.179993,31.100000,162.139999,14.56,110.800003,76.809998,...,33.259998,119.980003,252.690002,65.160004,52.430000,44.049999,19.950001,194.850006,18.290001,27.930000
2024-01-08,133.380005,31.730000,14.58,61.910000,185.559998,31.250000,161.429993,14.68,112.400002,77.120003,...,32.720001,121.970001,261.089996,66.650002,54.580002,44.540001,19.650000,196.149994,18.770000,28.049999
2024-01-09,130.679993,31.940001,14.38,61.389999,185.139999,31.549999,162.309998,14.79,112.730003,76.150002,...,32.689999,121.870003,256.440002,65.430000,54.349998,44.040001,19.500000,195.940002,18.209999,28.040001
